# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [6]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2009\\part.0.parque

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
print(dd_px.columns)
dd_px

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=89,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
AIT,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


In [ ]:
'''
First, Less Efficient Version:

dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_Lag_1 = x['Close'].shift(1))
)

dd_rets = dd_doubleshift.assign(
    Returns = lambda x: x['Close']/x['Close_Lag_1'] - 1
)

dd_adj_lag1_shift = dd_rets.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_Lag_1 = x['Adj Close'].shift(1))
)

dd_hi_lo = dd_adj_lag1_shift.assign(
    High_Low_Range = lambda x: x['High'] - x['Low']
)

dd_hi_lo.compute()
dd_feat = dd_hi_lo
print(dd_feat)

'''

C:\Users\irene\AppData\Local\Temp\ipykernel_22004\3579762283.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
C:\Users\irene\AppData\Local\Temp\ipykernel_22004\3579762283.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_adj_lag1_shift = dd_rets.groupby('ticker', group_keys=False).apply(


KeyboardInterrupt: 

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [57]:
'''
Second, More Efficient Version:
'''

dd_doubleshift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_Lag_1 = x['Close'].shift(1), 
                       Adj_Close_Lag_1 = x['Adj Close'].shift(1))
)
dd_feat = dd_doubleshift.assign(
    Returns = lambda x: x['Close']/x['Close_Lag_1'] - 1, 
    High_Low_Range = lambda x: x['High'] - x['Low']
)

dd_feat.compute()

C:\Users\irene\AppData\Local\Temp\ipykernel_22004\2636899747.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_doubleshift = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_Lag_1,Adj_Close_Lag_1,Returns,High_Low_Range
ticker,,,,,,,,,,,,,
ACN,2017-01-03,117.379997,117.809998,115.820000,116.459999,110.311638,2351600.0,ACN.csv,2017,NaN,NaN,NaN,1.989998
ACN,2017-01-04,116.910004,117.800003,116.430000,116.739998,110.576851,2639800.0,ACN.csv,2017,116.459999,110.311638,0.002404,1.370003
ACN,2017-01-05,116.980003,117.139999,114.949997,114.989998,108.919258,3685400.0,ACN.csv,2017,116.739998,110.576851,-0.014991,2.190002
ACN,2017-01-06,114.989998,116.739998,114.339996,116.300003,110.160088,4125300.0,ACN.csv,2017,114.989998,108.919258,0.011392,2.400002
ACN,2017-01-09,116.139999,116.349998,114.919998,115.000000,108.928719,2550800.0,ACN.csv,2017,116.300003,110.160088,-0.011178,1.430000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2004-06-28,8.440000,8.500000,7.660000,8.030000,8.030000,809200.0,ZIXI.csv,2004,8.660000,8.660000,-0.072748,0.840000
ZIXI,2004-06-29,8.060000,8.060000,7.590000,7.650000,7.650000,467500.0,ZIXI.csv,2004,8.030000,8.030000,-0.047322,0.470000
ZIXI,2004-06-30,7.640000,7.950000,7.550000,7.890000,7.890000,364000.0,ZIXI.csv,2004,7.650000,7.650000,0.031373,0.400000


In [58]:
print(dd_feat)

Dask DataFrame Structure:
                          Date     Open     High      Low    Close Adj Close   Volume  source   Year Close_Lag_1 Adj_Close_Lag_1  Returns High_Low_Range
npartitions=89                                                                                                                                          
ACN             datetime64[ns]  float64  float64  float64  float64   float64  float64  string  int32     float64         float64  float64        float64
AIT                        ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...            ...
...                        ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...            ...
ZIXI                       ...      ...      ...      ...      ...       ...      ...     ...    ...         ...             ...      ...            ...
ZIXI                       ...      ...      ...      ..

In [59]:
import pandas as pd
import dask.dataframe as dd
pd_feat = dd_feat.compute()
# pd_feat.head(5)
# print(isinstance(pd_feat, pd.DataFrame))

pd_feat['Rolling_Average_10'] = pd_feat['Returns'].groupby.rolling(10).mean()
pd_feat



AttributeError: 'function' object has no attribute 'rolling'

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not necessary to conver to pandas to calculate the moving average return, but it is simpler coding to convert it to pandas first, then calculate.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.